# net/http

se trata de uma biblioteca de interação com servidores via requests e responses

análogo ao httpx do python

temos os seguinte componentes e funcionalidades dentro dela:

# Constantes

basicamente compreende dois tipos:

## 1.  Métodos HTTP

MethodGet     = "GET"
MethodHead    = "HEAD"
MethodPost    = "POST"
MethodPut     = "PUT"
MethodPatch   = "PATCH" // RFC 5789
MethodDelete  = "DELETE"
MethodConnect = "CONNECT"
MethodOptions = "OPTIONS"
MethodTrace   = "TRACE"

## 2.   Status Code

### 1xx

StatusContinue           = 100 // RFC 9110, 15.2.1

StatusSwitchingProtocols = 101 // RFC 9110, 15.2.2

StatusProcessing         = 102 // RFC 2518, 10.1

StatusEarlyHints         = 103 // RFC 8297

### 2xx
StatusOK                   = 200 // RFC 9110, 15.3.1

StatusCreated              = 201 // RFC 9110, 15.3.2

StatusAccepted             = 202 // RFC 9110, 15.3.3

StatusNonAuthoritativeInfo = 203 // RFC 9110, 15.3.4

StatusNoContent            = 204 // RFC 9110, 15.3.5

StatusResetContent         = 205 // RFC 9110, 15.3.6

StatusPartialContent       = 206 // RFC 9110, 15.3.7

StatusMultiStatus          = 207 // RFC 4918, 11.1

StatusAlreadyReported      = 208 // RFC 5842, 7.1

StatusIMUsed               = 226 // RFC 3229, 10.4.1

### 3xx
StatusMultipleChoices  = 300 // RFC 9110, 15.4.1

StatusMovedPermanently = 301 // RFC 9110, 15.4.2

StatusFound            = 302 // RFC 9110, 15.4.3

StatusSeeOther         = 303 // RFC 9110, 15.4.4

StatusNotModified      = 304 // RFC 9110, 15.4.5

StatusUseProxy         = 305 // RFC 9110, 15.4.6

StatusTemporaryRedirect = 307 // RFC 9110, 15.4.8

StatusPermanentRedirect = 308 // RFC 9110, 15.4.9

### 4xx
StatusBadRequest                   = 400 // RFC 9110, 15.5.1

StatusUnauthorized                 = 401 // RFC 9110, 15.5.2

StatusPaymentRequired              = 402 // RFC 9110, 15.5.3

StatusForbidden                    = 403 // RFC 9110, 15.5.4

StatusNotFound                     = 404 // RFC 9110, 15.5.5

StatusMethodNotAllowed             = 405 // RFC 9110, 15.5.6

StatusNotAcceptable                = 406 // RFC 9110, 15.5.7

StatusProxyAuthRequired            = 407 // RFC 9110, 15.5.8

StatusRequestTimeout               = 408 // RFC 9110, 15.5.9

StatusConflict                     = 409 // RFC 9110, 15.5.10

StatusGone                         = 410 // RFC 9110, 15.5.11

StatusLengthRequired               = 411 // RFC 9110, 15.5.12

StatusPreconditionFailed           = 412 // RFC 9110, 15.5.13

StatusRequestEntityTooLarge        = 413 // RFC 9110, 15.5.14

StatusRequestURITooLong            = 414 // RFC 9110, 15.5.15

StatusUnsupportedMediaType         = 415 // RFC 9110, 15.5.16

StatusRequestedRangeNotSatisfiable = 416 // RFC 9110, 15.5.17

StatusExpectationFailed            = 417 // RFC 9110, 15.5.18

StatusTeapot                       = 418 // RFC 9110, 15.5.19 (Unused)

StatusMisdirectedRequest           = 421 // RFC 9110, 15.5.20

StatusUnprocessableEntity          = 422 // RFC 9110, 15.5.21

StatusLocked                       = 423 // RFC 4918, 11.3

StatusFailedDependency             = 424 // RFC 4918, 11.4

StatusTooEarly                     = 425 // RFC 8470, 5.2.

StatusUpgradeRequired              = 426 // RFC 9110, 15.5.22

StatusPreconditionRequired         = 428 // RFC 6585, 3

StatusTooManyRequests              = 429 // RFC 6585, 4

StatusRequestHeaderFieldsTooLarge  = 431 // RFC 6585, 5

StatusUnavailableForLegalReasons   = 451 // RFC 7725, 3

### 5xx
StatusInternalServerError           = 500 // RFC 9110, 15.6.1

StatusNotImplemented                = 501 // RFC 9110, 15.6.2

StatusBadGateway                    = 502 // RFC 9110, 15.6.3

StatusServiceUnavailable            = 503 // RFC 9110, 15.6.4

StatusGatewayTimeout                = 504 // RFC 9110, 15.6.5

StatusHTTPVersionNotSupported       = 505 // RFC 9110, 15.6.6

StatusVariantAlsoNegotiates         = 506 // RFC 2295, 8.1

StatusInsufficientStorage           = 507 // RFC 4918, 11.5

StatusLoopDetected                  = 508 // RFC 5842, 7.2

StatusNotExtended                   = 510 // RFC 2774, 7

StatusNetworkAuthenticationRequired = 511 // RFC 6585, 6


# Structs

## http.Client{}

realiza o gerenciamento de parametros a serem utilizados em um request por um Client

**é um agente configurável que podemos especificar como o request será feito**

não é obrigatoriamente necessário a criação de um Client ao usar a net/http já que esta fornece funções como `http.Get()` e `http.Post()` com o intuito de deixar a criação do Client em segundo plano

o seu uso é necessário e relevante em situações onde **se deseja maior controle sobre os requests HTTP feitos**

um exemplo de definção de http.Client{} é:

```terraform
package main

import (
	"fmt"
	"io"
	"net/http"
	"time"
)

func main() {
	// 1. Create a custom http.Client
	client := &http.Client{
		Timeout: 10 * time.Second, // Set a 10-second timeout for the entire request
		Transport: &http.Transport{
			MaxIdleConns:        10,
			IdleConnTimeout:     30 * time.Second,
			DisableKeepAlives:   false,
			TLSHandshakeTimeout: 5 * time.Second,
			// Add other transport configurations as needed
		},
		CheckRedirect: func(req *http.Request, via []*http.Request) error {
			fmt.Printf("Following redirect to: %s\n", req.URL)
			if len(via) >= 10 {
				return fmt.Errorf("stopped after 10 redirects")
			}
			return nil
		},
		// Jar: ... (for custom cookie management)
	}

	// 2. Create an HTTP request
	req, err := http.NewRequest("GET", "https://httpbin.org/get", nil)
	if err != nil {
		fmt.Println("Error creating request:", err)
		return
	}

	// Optionally add headers
	req.Header.Set("User-Agent", "MyCustomGoClient/1.0")

	// 3. Send the request using the custom client
	resp, err := client.Do(req)
	if err != nil {
		fmt.Println("Error making request:", err)
		return
	}
	defer resp.Body.Close()

	// 4. Process the response
	fmt.Println("Status Code:", resp.StatusCode)
	body, err := io.ReadAll(resp.Body)
	if err != nil {
		fmt.Println("Error reading response body:", err)
		return
	}
	fmt.Println("Response Body:", string(body))
}
```

entrando em detalhes de seus componentes

### **Transport**

como será estabelecida a conexão com o server (TCP/IP, UDP, TLS, etc)

### **Redirection Policy**

como será lidado os status code 3xx (de redirecionamento)

**serão seguidos ou um erro é retornado?**

**por padrão ele é redirecionado**

### **Cookies**

gerencia a politica de cookies, isto é, se serão enviados cookies junto dos requests

**por padrão ele envia cookies**

### **Timeouts**

seta o timeout de execução do request para diferentes partes do seu processo como:

+   conexão

+   response

+   etc

## http.CloseNotifier{}

## http.ConnState{}

## http.Cookie{}

## http.CookieJar{}

## http.Dir{}

## http.File{}

## http.FileSystem{}

## http.Flusher{}

## http.HTTP2Config{}

## http.Handler{}

## http.HandlerFunc{}

## http.Header{}

## http.Hijacker{}

## http.MaxBytesError{}

## http.Protocols{}

## http.ProtocolError{}

## http.PushOptions{}

## http.Pusher{}

## http.Request{}

## http.Response{}

## http.ResponseController{}

## http.ResponseWriter{}

## http.RoundTripper{}

## http.SameSite{}

## http.ServeMux{}

## http.Server{}

## http.Transport{}

# Funções

### CanonicalHeaderKey(s `string`) `string`

### DectectContentType(data `[]byte`) `string`

### Error(w `ResponseWriter`, error `string`, code `int`)

### Handle(pattern `string` handler `Handler`)

### HandleFunc(patter `string`, handler `func(ResponseWriter, *Request)`)

### ListAndServe(addr `string`, handler `Handler`) `error`

### ListAndServeTLS(addr, certFile, keyFile `string`, handler `Handler`) `error`

### MaxBytesReader(w `ResponseWriter`, r `io.ReadCloser`, n `int64`) `io.ReadCloser`

### NotFound(w `ResponseWriter`, r `*Request`)

### ParseHTTPVersion(vers `string`) (major, minor `int`, ok `bool`)

### ParseTime(text `string`) (t `time.Time`, err `error`)

### ProxyFromEnvironment(req `*Request`) (`*url.URL`, `error`)

### ProxyURL(fixedURL `*url.URL`) func(`*Request`) (`*url.URL`, `error`)

### Redirect(w `ResponseWriter`, r `*Request`, url `string`, code `int`)

### Serve(l `net.Listener`, handler `Handler`) `error`

### ServeContent(w `ResponseWriter`, req `*Request`, name `string`, modtime `time.Time`, content `io.ReadSeeker`)

### ServeFile(w `ResponseWriter`, r `*Request`, name `string`)

### ServeFileFS(w `ResponseWriter`, r `*Request`, fsys `fs.FS`, name `string`)

### ServeTLS(l `net.Listener`, handler `Handler`, certFile, keyFile `string`) `error`

### SetCookie(w `ResponseWriter`, cookie `*Cookie`)

### StatusCode(code `int`) `string`